In [6]:
import requests

In [96]:
class Rate:
    def __init__(self, format_='value', diff_= False):
        self.format = format_
        self.diff = diff_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            #Ниже добавлена проверка на diff, если True выводится не значение валюты, а разница между текущим и предыдущим значением
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            elif self.format == 'value' and self.diff == True:
                return (response[currency]['Value'] - response [currency]['Previous'])
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
            
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')
    
    def max_rate(self):
        """Метод возвращает имя валюты с наибольшим курсом и значение курса - по ДЗ - Задание 1"""
        check_value = 0
        for i,j in self.exchange_rates().items():
            if j['Value'] > check_value:
                Name_cur = i
                check_value = j['Value']
        return (Name_cur, check_value)

Задание №1

In [97]:
r = Rate ()
name_cur, value_cur = r.max_rate()
print ('Валюта {} имеет максимальный курс, равный {}'.format(name_cur, value_cur) )
#print (r.max_rate())

Валюта XDR имеет максимальный курс, равный 108.7264


Задание №2

In [98]:
r1 = Rate(format_ ='full', diff_=True )
r1.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 77.0284,
 'Previous': 77.9157}

In [99]:
r2 = Rate(format_ ='full')
r2.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 77.0284,
 'Previous': 77.9157}

In [100]:
r2 = Rate(format_ ='value', diff_=True)
r2.usd()

-0.8872999999999962

Задание №3
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

In [175]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [176]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [201]:
class Designer(Employee):
    def __init__(self, name, seniority, int_award=2):
        self.int_award = int_award
        super().__init__(name, seniority)
        #Устанавливаем первоначальные баллы с учетом лет и количества премий
        self.points = self.seniority+self.int_award*2
        if self.points//7 < 2:
            self.grade = 1
        else:
            self.grade = self.points//7

    def check_time_for_upgrade(self):
        # условие повышения сотрудника из презентации, изменено от демонстрации, так как при добавлении премии, может проскочит остаток 0
        if self.points // 7 > self.grade:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()            
            
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        # проверка повышения грейда вынесена в отдельный метод, так как грейд повышается и по аккредитациям и по премии
        # название метода оставлено идентично разработчику, для совместимости.
        self.seniority += 1
        self.points += 1
        self.check_time_for_upgrade()
        
    def new_awards (self):
        #добавляем премию, считаем, что добавляется по 1 и затем вызываем проверку в повышении грейда
        self.int_award += 1
        self.points += 2
        self.check_time_for_upgrade()

In [210]:
ivan = Designer(name='Иван', seniority=11)

In [211]:
ivan.check_if_it_is_time_for_upgrade()

Иван 2


In [214]:
ivan.new_awards()

Иван 3
